**Normalized coordinates are calculated in each 1m x 1m block.**

**NOT WORKING WELL**

In [11]:
import sys
sys.path
sys.path.append(r"C:\Users\yanpe\.env\Lib\site-packages")
print(sys.path)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_

In [12]:
import os
import torch
import torch.nn.functional as F
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, knn_interpolate
from torch_geometric.typing import WITH_TORCH_CLUSTER
from torch_geometric.data import Data

import open3d as o3d
import numpy as np

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [13]:
# Empty the CUDA cache
torch.cuda.empty_cache()

In [14]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointNetConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [15]:
class FPModule(torch.nn.Module):
    def __init__(self, k, nn):
        super().__init__()
        self.k = k
        self.nn = nn

    def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
        x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
        if x_skip is not None:
            x = torch.cat([x, x_skip], dim=1)
        x = self.nn(x)
        return x, pos_skip, batch_skip

In [16]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 6, 64, 64, 128])) # 3 (pos) + 6 (x)
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
        self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
        self.fp1_module = FPModule(3, MLP([128 + 6, 128, 128, 128]))

        self.mlp = MLP([128, 128, 128, num_classes], dropout=0.5, norm=None)

        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.lin3 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)

        fp3_out = self.fp3_module(*sa3_out, *sa2_out)
        fp2_out = self.fp2_module(*fp3_out, *sa1_out)
        x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

        return self.mlp(x).log_softmax(dim=-1)


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_classes=13).to(device)

In [38]:
# File path to the pre-trained model
script_dir = os.path.dirname(os.getcwd())
model_file_path= os.path.join(script_dir, '.', 'docs', 'pointnet2_s3dis_transform_seg_x6_27_checkpoint.pth')


In [39]:
# Load the checkpoint dictionary
checkpoint = torch.load(model_file_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

# print(model_state_dict)

In [40]:
model.load_state_dict(model_state_dict)
model.eval()

Net(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(9, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(134, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [21]:
# Pcd files
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.pcd"
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

In [22]:
def process_point_cloud(pcd, block_size=1.0):
    """
    Process a point cloud file and append block-wise normalized coordinates.
    
    Assumptions:
      - The point cloud file (e.g., .npy) contains an Nx6 array where each row is
        (x, y, z, r, g, b).
      - The function computes normalized coordinates per block. A block is defined
        as a cube of side `block_size` covering the extent of the point cloud.
      - The output array has shape (N, 9) where the extra 3 features (nx, ny, nz) are 
        the normalized coordinates within each block.
    
    Parameters:
      file_path (str): Path to the point cloud file.
      block_size (float): Side length for each block (in same units as the point coordinates).
    
    Returns:
      numpy.ndarray: Processed point cloud of shape (N, 9).
    """
    # Load the point cloud.
    
    points = np.array(pcd.points)  
    colors = np.array(pcd.colors)    
    
    # Compute the overall minimum bound for x, y, z
    min_bound = np.min(points, axis=0)
    
    # Compute block index for each point (each dimension)
    block_idx = np.floor((points - min_bound) / block_size).astype(np.int32)
    
    # Compute the lower bound for the block each point belongs to
    block_lower_bound = min_bound + block_idx * block_size
    
    # Calculate normalized coordinates within the block
    normalized_coords = (points - block_lower_bound) / block_size
    
    # Concatenate the original 6 features with the normalized coordinates (nx, ny, nz)
    processed_points = np.concatenate([points, colors, normalized_coords], axis=1)
    
    return processed_points



In [23]:
print(len(pcd.points))
print(len(pcd.colors))
print(len(pcd.normals))

35966772
35966772
0


In [24]:
# Downsample the point cloud with a voxel of 0.03
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

print(len(downpcd.points))

866885


In [26]:
o3d.visualization.draw_geometries([downpcd], point_show_normal=False)

In [30]:
processed_downpcd = process_point_cloud(downpcd, block_size=1.0)

# Convert the NumPy array to a PyTorch tensor.
processed_downpcd_tensor = torch.from_numpy(processed_downpcd)
processed_downpcd_tensor = processed_downpcd_tensor.float()

# Extract the first 3 features (e.g., x, y, z).
pos = processed_downpcd_tensor[:, :3]

# Extract the last 6 features (e.g., r, g, b, and the 3 normalized coordinates).
x = processed_downpcd_tensor[:, 3:]

In [31]:
# Create a Data object with x (6 features) and pos (coordinates)
data = Data(x=x, pos=pos)

#pre_transform = T.NormalizeScale()
#data = pre_transform(data)

In [88]:
import os
os.cpu_count()

22

In [32]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

num_workers = 20
batch_size = 32

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [41]:
model.eval()
with torch.no_grad():
    for data in custom_loader:
        data = data.to(device)
        predictions = model(data)
        labels = predictions.argmax(dim=-1)
        # Process the labels as needed
        labels_arr = labels.cpu().numpy()
        # Count occurrences of labels
        unique_labels, label_counts = np.unique(labels_arr, return_counts=True)
        # Combine and print
        result_labels = np.array(list(zip(unique_labels, label_counts)))
        print("Label counts:")
        print(result_labels)    



Label counts:
[[     0   6153]
 [     1  87202]
 [     2 189940]
 [     3  17127]
 [     4  12160]
 [     5     27]
 [     6  38937]
 [     7  17977]
 [     8   3727]
 [     9     46]
 [    10 415402]
 [    12  78187]]


batch_size 32, num_workers 16: prediction 7min

batch_size = 64, num_workers 16: prediction 5.5min

batch_size = 32, num_workers 20: prediction 5min

batch_size = 128, num_workers 20: prediction 7.5min

batch_size = 48, num_workers 20: prediction 7min



In [42]:

# Define the number of classes in your model's predictions
num_classes = 13  # Adjust based on your number of classes


# Define a fixed color map for 13 labels
color_map = np.array([
    [1.0, 0.0, 0.0],  # Label 0: Red,  'ceiling'
    [0.0, 1.0, 0.0],  # Label 1: Green, 'floor'
    [0.0, 0.0, 1.0],  # Label 2: Blue,  'wall'
    [1.0, 1.0, 0.0],  # Label 3: Yellow, 'beam'
    [1.0, 0.0, 1.0],  # Label 4: Magenta, 'column'
    [0.0, 1.0, 1.0],  # Label 5: Cyan, 'window'
    [0.5, 0.5, 0.5],  # Label 6: Gray, 'door'
    [1.0, 0.5, 0.0],  # Label 7: Orange, 'chair'
    [0.5, 0.0, 1.0],  # Label 8: Purple, 'table'
    [0.5, 1.0, 0.5],  # Label 9: Light Green, 'bookcase'
    [0.5, 0.5, 1.0],  # Label 10: Light Blue, 'sofa'
    [1.0, 0.5, 0.5],  # Label 11: Pink, 'board'
    [0.0, 0.0, 0.0]   # Label 12: Black, 'clutter'
])

In [43]:
# Assuming `labels` is the tensor containing predicted labels for each point
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]


In [36]:
print(len(predicted_colors))

866885


In [44]:
# Assuming `pcd` is your Open3D point cloud object
downpcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [45]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([downpcd])

In [67]:
# Save the point cloud to a file
save_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/labelled/Smartlab_aalto_pcd_ifctrained_bw_label_pointnet2_x6_0.02_20250412.ply"
o3d.io.write_point_cloud(save_path, downpcd)


True

In [23]:
del down_points
del down_colors
torch.cuda.empty_cache()